In [22]:
#import dependencies
import pandas as pd
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
#call up tsa data
tsa_data="Resources/tsa_data.csv"
tsa = pd.read_csv(tsa_data)
#change date formatting for consistency
tsa['Date']=pd.to_datetime(tsa['Date'].astype(str), format='%m/%d/%Y')
#changed numbers into floats
tsa["Total Traveler Throughput 2020"] = tsa["Total Traveler Throughput 2020"].str.replace(',','')
tsa["Total Traveler Throughput 2020"] = tsa["Total Traveler Throughput 2020"].astype(float)
tsa["Total Traveler Throughput 2019"] = tsa["Total Traveler Throughput 2019"].str.replace(',','')
tsa["Total Traveler Throughput 2019"] = tsa["Total Traveler Throughput 2019"].astype(float)
#reduce variance by using rolling mean for traveler numbers
tsa['Average 2019'] = tsa.iloc[:,2].rolling(window=7).mean()
tsa['Average 2020'] = tsa.iloc[:,1].rolling(window=7).mean()


In [3]:
#prepare tsa file for merge with NYT data
passenger_numbers_2020 = tsa.loc[:,["Date","Total Traveler Throughput 2020"]]

In [4]:
#change date to number string
numbered_tsa = passenger_numbers_2020
numbered_tsa["Date"]=numbered_tsa["Date"].astype(str)
numbered_tsa['Date']=numbered_tsa['Date'].str.replace('-','')
numbered_tsa['Date']=numbered_tsa['Date'].astype(float)
#limited dates to match with those from nyt dataset
numbered_tsa = numbered_tsa.loc[numbered_tsa["Date"]<20200729]

In [5]:
#load and setup NYT data into dataframe
nyt_data="Resources/covid_19_state_level_data.csv"
nyt = pd.read_csv(nyt_data) 

In [6]:
#edit nyt data to only include relevant info
curated_nyt = nyt.loc[:,["date","state","cases"]]
curated_nyt = curated_nyt.rename(columns={"date":"Date"})

In [7]:
#change date into float
numbered_nyt = curated_nyt
numbered_nyt["DateFormat"]=numbered_nyt["Date"]
numbered_nyt["DateFormat"]=numbered_nyt["DateFormat"].apply(lambda x:
                                             dt.datetime.strptime(x,'%Y-%m-%d'))
numbered_nyt['Date']=numbered_nyt['Date'].str.replace('-','')
numbered_nyt['Date']=numbered_nyt['Date'].astype(float)

In [8]:
#set up nyt dates to match tsa dates
tsa_numbered_nyt = numbered_nyt.loc[numbered_nyt["Date"]>20200230]

In [9]:
#group by state to get total case number for comparison and make new column of sums
tsa_numbered_nyt["Total Cases"] = tsa_numbered_nyt.groupby(["Date"])["cases"].transform("sum")
#make a new dataframe of just date and total cases and drop duplicates
nyt_nationwide = tsa_numbered_nyt.loc[:,["Date","DateFormat","Total Cases"]]
nyt_nationwide = nyt_nationwide.drop_duplicates()
#Make a new column of rate of change in total cases using .diff
nyt_nationwide["Case Rate of Change"]= nyt_nationwide["Total Cases"].diff()
nyt_nationwide['Averaged Rate of Change'] = nyt_nationwide.iloc[:,2].rolling(window=7).mean()

C:\Users\jekor\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
combined_total_data = pd.merge(numbered_tsa,nyt_nationwide, how="left",on="Date")
correlation_total_data = combined_total_data.loc[:,["Total Traveler Throughput 2020","Averaged Rate of Change"]]
for i in range(-30,31):
    correlation_total_data[i]=correlation_total_data["Averaged Rate of Change"].shift(i)

In [11]:
#make individual state dataframes for Georgia, California, Massachusetts, and Texas
#also create Case rate of change columns for all states
California = curated_nyt.loc[curated_nyt["state"]=="California"]
California["Case Rate of Change"]= California["cases"].diff()
California['Averaged Rate of Change'] = nyt_nationwide.iloc[:,4].rolling(window=7).mean()
Georgia = curated_nyt.loc[curated_nyt["state"]=="Georgia"]
Georgia["Case Rate of Change"]= Georgia["cases"].diff()
Georgia['Averaged Rate of Change'] = nyt_nationwide.iloc[:,4].rolling(window=7).mean()
Massachusetts = curated_nyt.loc[curated_nyt["state"]=="Massachusetts"]
Massachusetts["Case Rate of Change"]= Massachusetts["cases"].diff()
Massachusetts['Averaged Rate of Change'] = nyt_nationwide.iloc[:,4].rolling(window=7).mean()
Texas = curated_nyt.loc[curated_nyt["state"]=="Texas"]
Texas["Case Rate of Change"]= Texas["cases"].diff()
Texas['Averaged Rate of Change'] = nyt_nationwide.iloc[:,4].rolling(window=7).mean()

C:\Users\jekor\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jekor\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\jekor\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [12]:
airport_data = "Resources/covid_impact_on_airport_traffic.csv"
airport = pd.read_csv(airport_data)

In [13]:
#remove not relevant columns from airport data
us_airport_data = airport[airport["Country"]=="United States of America (the)"]

In [14]:
#continue removing not relevant columns
curated_airport_data = us_airport_data.loc[:,["Date","AirportName","State","PercentOfBaseline"]]

In [46]:
#create new column for date with datetime format
curated_airport_data['DateFormat']=curated_airport_data['Date']
curated_airport_data["DateFormat"]=curated_airport_data["DateFormat"].apply(lambda x:
                                             dt.datetime.strptime(x,'%Y-%m-%d'))
#set date column as number
curated_airport_data['Date']=curated_airport_data['Date'].str.replace('-','')
curated_airport_data['Date']=curated_airport_data['Date'].astype(float)

TypeError: strptime() argument 1 must be str, not float

In [16]:
curated_airport_data=curated_airport_data.sort_values(by=["Date"])

In [17]:
#test airport df
LAX = curated_airport_data.loc[curated_airport_data["AirportName"]=="Los Angeles International"]
LAX['Averaged POB'] = nyt_nationwide.iloc[:,3].rolling(window=7).mean()

In [18]:
#create rest of airport dfs
SFO = curated_airport_data.loc[curated_airport_data["AirportName"]=="San Francisco International"]
SFO['Averaged POB'] = nyt_nationwide.iloc[:,3].rolling(window=7).mean()
BOS =  curated_airport_data.loc[curated_airport_data["State"]=="Massachusetts"]
BOS['Averaged POB'] = nyt_nationwide.iloc[:,3].rolling(window=7).mean()
ATL =  curated_airport_data.loc[curated_airport_data["State"]=="Georgia"]
ATL['Averaged POB'] = nyt_nationwide.iloc[:,3].rolling(window=7).mean()
DFW =  curated_airport_data.loc[curated_airport_data["State"]=="Texas"]
DFW['Averaged POB'] = nyt_nationwide.iloc[:,3].rolling(window=7).mean()

In [19]:
#test with 1 airport and city for combined data set
LAXCA = pd.merge(California,LAX,how="inner",on="Date")

In [20]:
#finish rest of airport city merges
SFOCA = pd.merge(California,SFO,how="inner",on="Date")
BOSMA = pd.merge(Massachusetts,BOS,how="inner",on="Date")
ATLGA = pd.merge(Georgia,ATL,how="inner",on="Date")
DFWTX = pd.merge(Texas,DFW,how="inner",on="Date")

In [47]:
LAX.head()

,Date,AirportName,State,PercentOfBaseline,DateFormat
2405,20200316.0,Los Angeles International,California,98,2020-03-16
2416,20200317.0,Los Angeles International,California,99,2020-03-17
2497,20200318.0,Los Angeles International,California,89,2020-03-18
2441,20200319.0,Los Angeles International,California,88,2020-03-19
2329,20200320.0,Los Angeles International,California,84,2020-03-20
